In [9]:
import import_ipynb
from InsertsDelComparisons import sorted_users, np, df, INDICES, map_, pd, recipes_indices, indices_of_first_attempts_per_user

In [2]:
def print_user_recipes(index):
    print(sorted_users[index])
    a = np.where(df['user_id'] == sorted_users[index])
    a = a[0]
    for i in a:
        print(df['recipe'].iloc[i][:20], df['event_date'].values[i])
    print(map_[index])

print_user_recipes(INDICES[1][2])


5b2509947f7816000145fa13
Pasta bolognese
Ingr 2022-09-04 08:32:09.543609
Pasta bolognese
Ingr 2022-09-04 08:51:14.524511
Sausage, mash and ve 2022-09-04 08:56:44.910849
Sausage, mash and ve 2022-09-04 08:57:02.876889
Odds and ends pasta
 2022-09-04 09:01:40.320894
Odds and ends pasta
 2022-09-04 09:04:16.484970
[56, 58, 60]


In [3]:
actions = [None] * len(df)
for arrays in map_:
    actions[arrays[0]] = '1st recipe submitted'
for i in range(len(df)):
    if actions[i] is None:
        if i in recipes_indices:
            actions[i] = 'next recipe submitted'
        else:  
            actions[i] = 'revision'

In [4]:
import os
import pm4py
from pm4py.visualization.petri_net import  visualizer as pn_vis_factory
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.alpha import  algorithm as alpha_miner

def save_user_session_info_graph(csv_filepath, group_num):
    graph_data = csv_filepath
    dataframe = pd.read_csv(graph_data, sep=';')
    event_log = pm4py.format_dataframe(dataframe, case_id='case_id', activity_key='activity', timestamp_key='timestamp')
    event_log = pm4py.convert_to_event_log(dataframe)

    new_file = 'xes_format.xes'
    pm4py.write_xes(event_log, new_file)
    log = pm4py.read_xes(new_file)
    net, initial_marking, final_marking = pm4py.discover_petri_net_alpha(log)
    performance_dfg, start_activities, end_activities = pm4py.discover_performance_dfg(event_log)
    
    try:
        pm4py.save_vis_performance_dfg(performance_dfg, start_activities, end_activities, 'results/process mining/groupnum{n}.png'.format(n=group_num))
    except: print('user only has 1 recipe')

#pn_vis_factory(pn_vis_factory.apply(net, initial_marking, final_marking))

In [21]:
ACTIONS = [';1st recipe submitted;', ';revision;', ';second recipe submitted;']
def format_user_data(user_index, with_enumeration=False, case_id=1):
    recipe_indices = map_[user_index].copy()
    if user_index == 186: last_index_where_written = 1091
    else: last_index_where_written = indices_of_first_attempts_per_user[user_index + 1]
    recipe_indices.append(last_index_where_written)

    where_in_df = np.where(df['user_id'] == sorted_users[user_index])
    first_line = str(case_id) + ACTIONS[0] + df.iloc[where_in_df[0][0]]['event_date']

    lines = [first_line]

    for i, index in enumerate(range(recipe_indices[0]+1, recipe_indices[-1]+1)):
        j = index
        is_new_recipe = index in recipe_indices
        while not is_new_recipe:
            line = ACTIONS[1] + df.iloc[where_in_df[0][i+1]]['event_date']
            lines.append(line)
            j+=1
            if j in recipe_indices: is_new_recipe = True

        try:
            if index in recipe_indices : lines.append(ACTIONS[2] + df.iloc[where_in_df[0][i+1]]['event_date'])
        except: continue
        
    result = lines
    index = 1
    for i, line in enumerate(lines):
        j = index if with_enumeration else 1
       
        if line.startswith(ACTIONS[1]):
            result[i] = str(case_id) + line
        elif line.startswith(ACTIONS[2]):
            index+=1
            j = index if with_enumeration else 1
            second_recipe_already_submitted = False
            for l in lines[:i]:
                if ACTIONS[2] in l : second_recipe_already_submitted = True
            if second_recipe_already_submitted:
                result[i] = str(case_id) + line.replace('second', 'third')
            else : result[i] = str(case_id) + line

    return  pd.DataFrame(sorted(set(result), key=result.index), columns=['case_id;activity;timestamp'])

In [22]:
for i in range(0,5):
    dframes = []
    for j, user in enumerate(INDICES[i]):
        A = format_user_data(user_index=user, case_id=j+1)
        dframes.append(A)

    res = pd.concat(dframes)
    path = 'results/process mining/group{i}.csv'.format(i=i+1)
    res.to_csv(path, index=False)
    save_user_session_info_graph(csv_filepath=path, group_num=i+1)


parsing log, completed traces :: 100%|██████████| 40/40 [00:00<00:00, 2928.78it/s]


In [8]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.alpha import  algorithm as alpha_miner
from pm4py.visualization.petri_net import visualizer as pn_vis_factory

""" performance_dfg, start_activities, end_activities = pm4py.discover_performance_dfg(log)
pm4py.save_vis_performance_dfg(performance_dfg, start_activities, end_activities, 'perf_dfg2.png')



net, im, fm = alpha_miner.apply(log)
gv = pn_vis_factory.apply(net, im, fm)
pn_vis_factory.view(gv) """
print(sorted_users[INDICES[1][2]])


5b2509947f7816000145fa13
